In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    print(list_of_files_in_genome_folder)
    if "Temp" in list_of_files_in_genome_folder:
        subprocess.run(f'rm -r "{genome_location}/{species}/Temp"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{genome_location}/{species}/Temp")
    return(genome_file)  

In [3]:
def create_query_folder(annotated_species_location,gene_name,species):
    import time


    list_of_files = os.listdir(annotated_species_location)
    if species in list_of_files:
        subprocess.run(f'rm -r "{annotated_species_location}/{species}"', shell = True, stderr = subprocess.DEVNULL)
    time.sleep(2.5)
    os.mkdir(f"{annotated_species_location}/{species}")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts/{gene_name}")
    
    

In [8]:
import os
import subprocess
from Bio import SeqIO

scaffold = ""
gene_name = "XM_013306595.1"
species = "Papilio_xuthus"

genome_location = f"/mnt/f/Genomes_2023-12-26"
genome_file = get_genome_file(genome_location,species)
annotated_species_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species"
query_species = species
create_query_folder(annotated_species_location,gene_name,species)
list_of_files = os.listdir(f"{genome_location}/{species}")
print(list_of_files)
for file_names in list_of_files:
    if file_names.endswith("_genomic.gff"):
        gff3_file = file_names

with open(f"{genome_location}/{species}/{gff3_file}") as gff_file_open:
    gff_file_list = gff_file_open.readlines()
complement = ''
exon_count = 1
gene_found = 0
exon_dictionary = {}
for line in gff_file_list:
    if gene_name in line:
        if "CDS" in line.split("\t"):

            print(line)
            gene_found = 1
            if complement == '':
                complement = line.split("\t")[6]
            exon_start = line.split("\t")[3]
            exon_stop = line.split("\t")[4]
            scaffold = line.split("\t")[0]
            exon_dictionary[f"Exon_{exon_count}"] = [exon_start,exon_stop]
            exon_count += 1
        if "exon" in line.split("\t") and gene_found == 1:
            break
print(complement)
print(exon_dictionary)
if complement == "+":
    exon_counter = 1
elif complement == "-":
    exon_counter = len(exon_dictionary)
else:
    print("Error!!")
    assert False

# list_of_query_species = os.listdir(query_location)
# if query_species not in list_of_query_species:
#     os.mkdir(f"{annotated_species_location}/{query_species}")
# list_of_folders_here = os.listdir(f"{annotated_species_location}/{query_species}")
# if "Transcripts" not
#     os.mkdir(f"{annotated_species_location}/{query_species}/{gene_name}")

print_out = ''
print("exon counter", exon_counter)        
for i in range(1,len(exon_dictionary)+1):
    
    exon_start, exon_stop = exon_dictionary[f"Exon_{exon_counter}"]
#     print(exon_start,exon_stop)
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{exon_start}-{exon_stop} > "{genome_location}/{species}/Temp/temp_gene.fa"', shell = True)
    exon_sequence_file = SeqIO.parse(f"{genome_location}/{species}/Temp/temp_gene.fa", "fasta")
    sequence = ''
    
    for sequences in exon_sequence_file:
#         print(sequences.id)
        if complement == "-":
            sequence = sequences.seq.reverse_complement() 
            
#             print(sequences.seq.reverse_complement().translate())
        else:
            sequence = sequences.seq
#             print(sequences.seq)
#             print(sequences.seq.translate())
        print_out += str(len(sequence))+"\t"
        output = f">{query_species}_{gene_name}_Exon{i}\n{sequence}"
    with open(f"{annotated_species_location}/{query_species}/Transcripts/{gene_name}/Exon{i}.fa", 'w') as outfile:
        outfile.write(output)
    print(output)
    if complement == "-":
        exon_counter -= 1
    else:
        exon_counter += 1
                                            
#         if gene_location in line.split("\t"):
#           print(line)
#           break
print(print_out)

['GCF_000836235.2_Pxut_1.1_genomic.fna', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nhr', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nin', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nsq', 'GCF_000836235.2_Pxut_1.1_genomic.gff']
['GCF_000836235.2_Pxut_1.1_genomic.fna', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nhr', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nin', 'GCF_000836235.2_Pxut_1.1_genomic.fna.nsq', 'GCF_000836235.2_Pxut_1.1_genomic.gff', 'Temp']
NW_013530472.1	Gnomon	CDS	3081817	3081905	.	+	0	ID=cds-XP_013162049.1;Parent=rna-XM_013306595.1;Dbxref=GeneID:106113722,GenBank:XP_013162049.1;Name=XP_013162049.1;gbkey=CDS;gene=LOC106113722;product=period circadian protein isoform X5;protein_id=XP_013162049.1

NW_013530472.1	Gnomon	CDS	3082038	3082115	.	+	1	ID=cds-XP_013162049.1;Parent=rna-XM_013306595.1;Dbxref=GeneID:106113722,GenBank:XP_013162049.1;Name=XP_013162049.1;gbkey=CDS;gene=LOC106113722;product=period circadian protein isoform X5;protein_id=XP_013162049.1

NW_013530472.1	Gnomon	CDS	3083000	3083166	

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Papilio_xuthus_XM_013306595.1_Exon1
ATGGACAATATGGATGACTCGGAGAACAATGCGAAGATATCAGATTCAGCGTACTCAAACAGTTGCAGCAACAGCCAGTCTAGGCGCAG


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Papilio_xuthus_XM_013306595.1_Exon2
ctCGAAATCAACTCACTCCGGAAGCAATTCGAGTGGCAGCAGCGGCTATGGCGGGAAACCATCTACATCCGGTTACAG


...

>Papilio_xuthus_XM_013306595.1_Exon23
gTGTGCGTTACATCCGagcttatttataaatatcaaattcaaACGAAGAGTATGGAGGAAGTCTTGTCTTGTGATAAGCAGAAATTAAAGCAATTGGAACAG


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Papilio_xuthus_XM_013306595.1_Exon24
ccATCGCTGGTGAATGAGCAATTGGGTCAGCTATACTTAGACCTTCAATTAGAAGGTGTGGCATCGAGATTGACACTTGAAGAGGGTGTCACGAGTTCCAGTAGTTCCGGAGAAGAGACCACTGTGAGAGTTCCCAAG


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Papilio_xuthus_XM_013306595.1_Exon25
GCGCCTAGGAAGAAACGGGAATACAGCAAATTGGTGATGATATATGAAGAGAACGCACCCCTCCCACCCCCTGACTCACCCGCTGCGGCTTCTGTGTCATAA
89	78	167	104	75	132	76	204	229	112	120	150	119	96	135	193	148	99	114	167	192	96	102	138	102	
